# MI2 Survey Data Indicator Scripting
This notebook contains scripts for munging data from the MI2 survey and calculating indicator values. Where applicable, indicator values are also caluclated per respondent.

**Contents**
* [Data Munging Script](#Data-Munging-Script)
* [Indicators](#Indicators)
 - [Indicator 1.1a](#Indicator-1.1a)
 - [Indicator 1.1b](#Indicator-1.1b)
 - [Indicator 1.1c](#Indicator-1.1c)
 - [Indicator 1.1](#Indicator-1.1)
 - [Indicator 4.1](#Indicator-4.1)
 - [Indicator 5.1](#Indicator-5.1)
 - [Indicator 6.1](#Indicator-6.1)
 

## Data Munging Script
This script codes survey questions and concatenates identically coded questions into a single column. Where multiple responses are allowed, responses will be separated by a comma. A cleaned spreadsheet with human readable responses is saved out as `clean_df.csv` and a coded spreadsheet is saved as `coded_df.csv`.

In [1]:
import pandas as pd
import numpy as np

raw_coded = pd.read_excel(r'C:\Users\Erik\OneDrive - Environmental Incentives\Downloads\mi2-data.xlsx', 
                          'RAW_CODED', 
                          header=None, # can't read duped headers
                          engine='openpyxl', 
                          date_parser=[3,4],
                          na_values='.')
codes = pd.read_excel(r'C:\Users\Erik\OneDrive - Environmental Incentives\Downloads\mi2-data-xwalk.xlsx', 
                      'xwalk', 
                      engine='openpyxl',
                     index_col='original')

recodes = pd.read_excel(r'C:\Users\Erik\OneDrive - Environmental Incentives\Downloads\mi2-data-xwalk.xlsx', 
                      'recodes', 
                      engine='openpyxl',
                      index_col='column')

# rename headers (can't read in duplicate headers, allowed for rename)
raw_coded.columns = codes['coded']
raw_coded = raw_coded.drop(0, axis=0)
raw_coded = raw_coded.dropna(how='all')

In [2]:
# concatenate identical questions and drop response row
columns = raw_coded.columns
column_filters = [columns.get_loc(c) for c in columns]  # -> int if unique else boolean array 

series = []
already_seen = set()
for i in column_filters:
    if type(i)==int:
        series.append(raw_coded.iloc[:,i])
    elif type(i)==np.ndarray:
        s = raw_coded.iloc[:,i].copy()
        name = s.columns[0]
        if name not in already_seen:
            s = s.apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)
            s.name = name
            series.append(s)
            already_seen.add(name)
clean_df = pd.concat(series, axis=1)

# convert all empty cells to nan
clean_df = clean_df.replace(r'^\s*$', np.nan, regex=True)

# convert responses to yes/no when multiple values allowed
multi_cols = ['BD3-1str', 'BD3-1prj', 'BD3-1ad', 'BD3-1asu', 'BD3-1imp', 'BD3-1evl']
for col in multi_cols:
    clean_df[col] = clean_df[col].fillna('No')
    clean_df[col] = clean_df[col].apply(lambda x: 'Yes' if x != 'No' else 'No')

clean_df.to_csv('clean_df.csv', index=False)

In [3]:
# code categorical columns
coded_df = clean_df.copy()

# specify dictionaries for encoding
yes_no = {
    'Yes': 1,
    'No': 0,
    'Not sure': 2
}

roles = {
    'I was part of a team responsible for managing a biodiversity program or programming biodiversity funds (Biodiversity Practitioner and/or A/COR)': 1,
    'I was part of a team responsible for managing a biodiversity program or programming biodiversity funds (Biodiversity Practitioner or A/COR)': 1,
    'I advised and/or provided technical assistance to those managing biodiversity programs or programming biodiversity funds (Biodiversity Advisor)': 2,
    'I engaged with biodiversity programming through facilitating strategic planning, procurement, and/or M&E and reporting (Program Officer)': 3,
    'I facilitated strategic planning, procurement, and/or M&E and reporting (Program Officer)': 3,
    'I worked in a different sector at USAID and supported one or more programs that included cross-sectoral integration with biodiversity (Partner in Integration)': 4,
    'I work in a different sector at USAID and supported cross-sectoral integration with biodiversity in a project or activity (Partner in Integration)': 4,
    'Other (please specify)': 5
}

roles_short = {
    'Strategy (CDCS)': 1,
    'Project design (PAD)': 2,
    'Activity design and/or procurement': 3,
    'Activity start up': 4,
    'Activity implementation': 5,
    'Evaluation': 6,
    'Other (please specify)': 7
}

op_units = {
    'Office of Forestry and Biodiversity': 1,
    'Other Washington Office or Operating Unit': 2,
    'Mission (please specify)': 3
}

likert = {
    'Strongly disagree': 1,
    'Disagree': 2,
    'Neither agree nor disagree':3,
    'Agree': 4,
    'Strongly Agree': 5
}

importance = {
    'Not important': 1,
    'Slightly important': 2,
    'Moderately important': 3,
    'Important': 4,
    'Very important': 5,
    "Don't know": 6
}

familiar = {
    'Not at all familiar': 1,
    'Somewhat familiar': 2,
    'Very familiar': 3
}

activities = {
    'strategy': 1,
    'project design': 2,
    'activity design': 3,
    'activity start up': 4,
    'activity implementation': 5,
    'evaluation': 6,
    'other': 7
}

effects = {
    'large positive effect': 1,
    'moderate positive effect': 2,
    'little effect': 3,
    'negative effect': 4,
    "don't know": 5
}

recode_dict = {
    'yes_no': yes_no,
    'roles': roles,
    'roles_short': roles_short,
    'op_units': op_units,
    'likert': likert,
    'importance': importance,
    'familiar': familiar,
    'activities': activities,
    'effects': effects
}

# code
for col in recodes.index:
    coded_df[col] = coded_df[col].map(recode_dict.get(recodes.loc[col][0]))

# code MI6.1-3 based on whether the text contains the key
def replace_if_contains(x):
    for key in effects.keys():
        if key in str(x).lower():
            return effects[key]

coded_df['MI6.x'] = coded_df['MI6.x'].apply(replace_if_contains)
    
# save to csv
coded_df.to_csv('coded_df.csv', index=False)

## Indicators
Indicator values are calculated per respondent where necessary and concatenated to `clean_df.csv`. Results are saved out after all indicators are calculated.

### Indicator 1.1a
**Shared understanding of the value of AM for biodiversity programming**					
This sub-indicator is calculated as the percentage of respondents that agrees or strongly agrees that AM improves biodiversity outcomes converted to a 5-point scale.

In [4]:
clean_df['i1_1a_values'] = clean_df['AM6-3']
clean_df['i1_1a'] = clean_df['i1_1a_values'].isin(['Strongly Agree', 'Agree'])
filt = clean_df['i1_1a_values'].isna()
clean_df['i1_1a'] = clean_df['i1_1a'].mask(filt, np.nan)

In [5]:
clean_df['i1_1a_values'].value_counts().plot(kind='pie', autopct='%1.1f%%', title='AM improves biodiversity outcomes')

<AxesSubplot:title={'center':'AM improves biodiversity outcomes'}, ylabel='i1_1a_values'>

In [6]:
n = clean_df['i1_1a'].count()
i = clean_df['i1_1a'].mean()
print(f'n: {n}, {i: .2%}')

n: 62,  82.26%


### Indicator 1.1b
**Shared understanding of the tools and practices recommended for AM in biodiversity programming converted to a 5-point scale.**					
This sub-indicator is calculated as the percentage of respondents that agrees or strongly agrees that the use of the four AM practices identified in the survey are important or very important in biodiverisity programming.					
*Note: if the respondent does not respond to any of the 4 prompts, they are excluded.*

In [7]:
response_count = clean_df[['PR5-5', 'PR6-5', 'PR7-4', 'PR8-5']].apply(pd.Series.value_counts, axis=1)
clean_df['i1_1b_values'] = response_count[['Important', 'Very important']].sum(axis=1)
clean_df['i1_1b'] = clean_df['i1_1b_values'] == 4
# overwrite with na if any responses are missing
filt = clean_df[['PR5-5', 'PR6-5', 'PR7-4', 'PR8-5']].isna().any(axis=1)
clean_df['i1_1b'] = clean_df['i1_1b'].mask(filt, np.nan)

In [8]:
clean_df['i1_1b_values'].value_counts().plot(kind='pie', autopct='%1.1f%%', title='Number of responses "Important" or "Very important"')

<AxesSubplot:title={'center':'Number of responses "Important" or "Very important"'}, ylabel='i1_1b_values'>

In [9]:
n = clean_df['i1_1b'].count()
i = clean_df['i1_1b'].mean()
print(f'n: {n}, {i: .2%}')

n: 55,  83.64%


## Indicator 1.1c
**Commitment to resourcing AM in biodiveristy programming**				
This sub-indicator is calculated as the percentage of respondents that agrees or strongly agrees that environment teams have access to the resources needed to adaptively manage their programs converted to a 5-point scale.					


In [10]:
clean_df['i1_1c_values'] = clean_df['AM_RES']
clean_df['i1_1c'] = clean_df['i1_1c_values'].isin(['Strongly Agree', 'Agree'])
# overwrite null values
filt = clean_df['i1_1c_values'].isna()
clean_df['i1_1c'] = clean_df['i1_1c'].mask(filt, np.nan)

In [11]:
clean_df['i1_1c_values'].value_counts().plot(kind='pie', autopct='%1.1f%%', title='environment teams have access to the resources needed to adaptively manage')

<AxesSubplot:title={'center':'environment teams have access to the resources needed to adaptively manage'}, ylabel='i1_1c_values'>

In [12]:
n = clean_df['i1_1c'].count()
i = clean_df['i1_1c'].mean()
print(f'n: {n}, {i: .2%}')

n: 59,  59.32%


## Indicator 1.1
**Sector-wide shared understanding index**				
This is a composite indicator calculated as the sum of three sub-indicators associated with a shared understanding of AM (range 3-15).					


In [13]:
bins = np.array([0, 0.2, 0.4, 0.6, 0.8, 1])
labels = [1, 2, 3, 4, 5]
means_1_1 = clean_df[['i1_1a', 'i1_1b', 'i1_1c']].mean()
scores_1_1 = pd.cut(means_1_1, bins=bins, labels=labels, right=False)  # right exclusive
i1_1 = scores_1_1.astype(int).sum()
print(f'Indicator 1.1 = {i1_1}')

Indicator 1.1 = 13


## Indicator 4.1
**Percentage of respondents reporting that enabling conditions for EBP are in place**				
This indicator is calculated as the percentage of respondents assigning scores of 4 or 5 to both EBP enabling conditions identified in the survey. The sample only includes respondents who assessed both EBP enabling conditions.					


In [14]:
agree_list = ['Agree', 'Strongly Agree']
filt = (clean_df['EV_KSA'].isin(agree_list))\
    & (clean_df['EV_RES'].isin(agree_list))
clean_df['i4_1'] = filt

# overwrite with na if any responses are missing
filt = (clean_df['EV_KSA'].isna()) \
    & (clean_df['EV_RES'].isna())
clean_df['i4_1'] = clean_df['i4_1'].mask(filt, np.nan)

In [15]:
clean_df['i4_1'].value_counts().plot(kind='pie', autopct='%1.1f%%', title='1.0: Assigned 4 or 5 to both conditions')

<AxesSubplot:title={'center':'1.0: Assigned 4 or 5 to both conditions'}, ylabel='i4_1'>

In [16]:
n = clean_df['i4_1'].count()
i = clean_df['i4_1'].mean()
print(f'n: {n}, {i: .2%}')

n: 58,  39.66%


## Indicator 5.1
**Percentage of respondents reporting broad uptake of AM practices in biodiversity programming**			
This indicator is calculated as the percentage of respondents reporting a high level of use across all four practices identified in the survey. The sample only includes respondents who assessed all four practices.

In [17]:
response_count = clean_df[['PR1-5', 'PR2-4', 'PR3-4', 'PR4-3']].apply(pd.Series.value_counts, axis=1)
clean_df['i5_1_values'] = response_count[['Agree', 'Strongly Agree']].sum(axis=1)
clean_df['i5_1'] = clean_df['i5_1_values'] == 4

# overwrite with na if any responses are missing
filt = clean_df[['PR1-5', 'PR2-4', 'PR3-4', 'PR4-3']].isna().any(axis=1)
clean_df['i5_1'] = clean_df['i5_1'].mask(filt, np.nan)

In [18]:
clean_df['i5_1_values'].value_counts().plot(kind='pie', autopct='%1.1f%%', title='Number of responses "Agree" or "Strongly Agree"')

<AxesSubplot:title={'center':'Number of responses "Agree" or "Strongly Agree"'}, ylabel='i5_1_values'>

In [19]:
n = clean_df['i5_1'].count()
i = clean_df['i5_1'].mean()
print(f'n: {n}, {i: .2%}')

n: 55,  56.36%


## Indicator 6.1
**Percentage of respondents reporting that application of MI/MI2 practices had a positive effect on Program Cycle outputs**				
This indicator is calculated as the percentage of respondents reporting that MI/MI2 practices had a moderate or strong poistive effect on the Program Cycle output for which they received techncial assistance. The sample excludes respondents who received TA but reported that they did not know the effect of MI/MI2 practices on the Program Cycle output.

In [20]:
clean_df['i6_1_values'] = clean_df['MI6.x']
clean_df['i6_1'] = clean_df['i6_1_values'].str.contains('positive').astype(float)
# mask instances of 'I don't know' note unicode character for apostrophe
mask = clean_df['i6_1_values']=='I don’t know'
clean_df['i6_1'] = clean_df['i6_1'].mask(mask, np.nan)

In [21]:
clean_df['i6_1'].value_counts().plot(kind='pie', autopct='%1.1f%%', title='1.0: MI2 had a positive effect')

<AxesSubplot:title={'center':'1.0: MI2 had a positive effect'}, ylabel='i6_1'>

In [22]:
n = clean_df['i6_1'].count()
i = clean_df['i6_1'].mean()
print(f'n: {n}, {i: .2%}')

n: 40,  95.00%


In [23]:
clean_df.to_csv('clean_df.csv', index=False)

In [24]:
coded_df = coded_df.merge(clean_df[['i1_1a', 'i1_1b',
       'i1_1c', 'i4_1', 'i5_1', 'i6_1']], 
               left_index=True, 
               right_index=True)

In [25]:
coded_df.to_csv('coded_df.csv', index=False)